# Data Preprocessing + Modelling

In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #please put your GPU"

In [2]:
import os
import re
import h5py
import pickle

import evaluate

import numpy as np
import tensorflow as tf

np.random.seed(2021)
tf.random.set_seed(2021)

import pandas as pd
import keras
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, classification_report, log_loss, confusion_matrix

import nltk
from nltk.stem.wordnet import WordNetLemmatizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Flatten
from keras.layers import Dropout, Conv1D, GlobalMaxPool1D, GRU, GlobalAvgPool1D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
df = pd.read_csv('data/data_after_preprocessing.csv')

In [4]:
df['label'].value_counts()

0    7221
1    2017
Name: label, dtype: int64

## Text Preprocessing

In [5]:
# Remove punctuation
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
df['tweet'].head()

c:\users\flyxs\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


0    service connected covid19 pandemic impacting t...
1    im not gone lie ion like normal girls i like e...
2    content warnings for billies documentary  stro...
3    why am i helping my suicidal irl im literally ...
4    the polluter pays principle is a threat to thi...
Name: tweet, dtype: object

In [6]:
# Lemmatization
nltk.download('wordnet')

df['tweet'] = df['tweet'].apply(lambda x:' '.join(WordNetLemmatizer().lemmatize(i) for i in x.split(' ')))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flyxs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df['tweet'].head()

0    service connected covid19 pandemic impacting t...
1    im not gone lie ion like normal girl i like em...
2    content warning for billy documentary  strong ...
3    why am i helping my suicidal irl im literally ...
4    the polluter pay principle is a threat to this...
Name: tweet, dtype: object

In [8]:
df.dtypes

tweet              object
label               int64
day                 int64
nlikes              int64
nreplies            int64
nretweets           int64
reply_to            int64
url                 int64
join_time           int64
tweets              int64
following           int64
followers           int64
likes               int64
media               int64
day_after           int64
tweet_length        int64
tweet_sentiment     int64
bio_sentiment       int64
first_person        int64
second_person       int64
third_person        int64
dtype: object

### Train test split

In [9]:
data_y = df['label']
data_x = df['tweet']

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, stratify=data_y, random_state = 2021)

In [10]:
x_train.apply(lambda x : len(x.split(' '))).quantile(0.95)

53.0

### Tokenizer

In [11]:
# Run keras Tokenizer
# Tokenize the sentences
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(x_train)
x_train_tok = tokenizer.texts_to_sequences(x_train)
x_test_tok = tokenizer.texts_to_sequences(x_test)

In [12]:
import pickle

with open('LSTM_models/tokenizer.pkl','wb') as f:
    pickle.dump(tokenizer,f)

### Pad sequences

In [13]:
#training constants
MAX_SEQ_LEN = 53 # Based on above

In [14]:
train_text_vec = pad_sequences(x_train_tok, maxlen=MAX_SEQ_LEN)
test_text_vec = pad_sequences(x_test_tok, maxlen=MAX_SEQ_LEN)

In [15]:
print('Number of Tokens:', len(tokenizer.word_index))
print("Max Token Index:", train_text_vec.max(), "\n")

print('Sample Tweet Before Processing:', x_train.values[0])
print('Sample Tweet After Processing:', tokenizer.sequences_to_texts([train_text_vec[0]]), '\n')

print('What the model will interpret:', train_text_vec[0].tolist())

Number of Tokens: 14216
Max Token Index: 14216 

Sample Tweet Before Processing: we can at least prevent people from such suicidal act  using coronil remains their choice eventually marne ke baad coronil kaam ka nahi bol ke kya fayda
Sample Tweet After Processing: ['we can at least prevent people from such suicidal act using coronil remains their choice eventually marne ke baad coronil kaam ka nahi bol ke kya fayda'] 

What the model will interpret: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 45, 43, 358, 788, 27, 42, 296, 1, 372, 470, 2691, 2146, 79, 768, 1153, 4575, 1957, 4576, 2691, 4577, 1958, 3629, 4578, 1957, 4579, 4580]


In [16]:
# One Hot Encode Y values:
encoder = LabelEncoder()

y_train_label = encoder.fit_transform(y_train.values)
y_train_label = to_categorical(y_train_label) 

y_test_label = encoder.fit_transform(y_test.values)
y_test_label = to_categorical(y_test_label) 

### Get class weights for the training data, this will be used in training

In [17]:
from collections import Counter

ctr = Counter(y_train.values)
print('Distribution of Classes:', ctr)

Distribution of Classes: Counter({0: 5776, 1: 1614})


In [18]:
# get class weights for the training data, this will be used data
y_train_int = np.argmax(y_train_label,axis=1)
cws_raw = class_weight.compute_class_weight('balanced', np.unique(y_train_int), y_train_int)
label = [0,1]

cws = dict(zip(label, cws_raw))

print(cws)

{0: 0.6397160664819944, 1: 2.2893432465923174}


c:\users\flyxs\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


## Modelling

## Naive Bayes

In [19]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
my_stop_words = text.ENGLISH_STOP_WORDS.union(['suicidal'])
count_vectorizer = CountVectorizer(min_df=0.01, stop_words=my_stop_words)
count = CountVectorizer(min_df=0.01, stop_words=my_stop_words)
x_train_2 = count.fit_transform(x_train)
x_test_2 = count.transform(x_test)

from sklearn import naive_bayes

mnb = naive_bayes.MultinomialNB()
mnb.fit(x_train_2, y_train)
y_pred_proba = mnb.predict_proba(x_test_2)
# dic = evaluate.threshold(y_pred_proba,y_test)
# print('Selected threshold: ', dic['threshold'])
print(evaluate.performance(y_test, mnb.predict(x_test_2),y_pred_proba)['report'])


The evaluation report of classification is:
Confusion Matrix:
[[1356   89]
 [ 272  131]]
Accuracy: 0.8046536796536796
Precision: 0.5954545454545455
Recall: 0.3250620347394541
F2 Score: 0.3575327510917031
AUC Score: 0.755799496853186



In [20]:
neg_class_keyword = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_keyword = mnb.feature_log_prob_[1, :].argsort()[::-1]
print(np.take(count.get_feature_names(),neg_class_keyword[:10]))
print(np.take(count.get_feature_names(),pos_class_keyword[:10]))

['wa' 'people' 'thought' 'like' 'just' 'im' 'dont' 'suicide' 'amp' 'know']
['im' 'thought' 'wa' 'just' 'like' 'dont' 'feel' 'tw' 'want' 'really']


### Bidirectional LSTM

In [21]:
DEFAULT_BATCH_SIZE = 64
DEFAULT_EPOCHS = 100

model = Sequential()
model.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits = False),
              metrics = ['accuracy']
             )

early_stop = keras.callbacks.EarlyStopping(monitor = 'loss',
                                              verbose = 1,
                                              patience = 10,
                                              mode = 'auto',
                                              restore_best_weights = True
                                             )

checkpoint = ModelCheckpoint('LSTM_models/best_LSTM_model.h5', monitor='loss', mode='auto', 
                             verbose = 1, save_best_only=True)

callbacks_list = [checkpoint, early_stop]

model.fit(x=train_text_vec,
          y=y_train_label,
          class_weight=cws,
          batch_size=DEFAULT_BATCH_SIZE,
          epochs=DEFAULT_EPOCHS,
          callbacks=callbacks_list,
          verbose=1,
          validation_data=(
              test_text_vec,
              y_test_label,
          ))

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/100
116/116 [==============================] - 33s 256ms/step - loss: 0.6364 - accuracy: 0.5298 - val_loss: 0.5126 - val_accuracy: 0.7489

Epoch 00001: loss improved from inf to 0.56433, saving model to LSTM_models\best_LSTM_model.h5
Epoch 2/100
116/116 [==============================] - 33s 281ms/step - loss: 0.3934 - accuracy: 0.8104 - val_loss: 0.3946 - val_accuracy: 0.8263

Epoch 00002: loss improved from 0.56433 to 0.38446, saving model to LSTM_models\best_LSTM_model.h5
Epoch 3/100
116/116 [==============================] - 33s 283ms/step - loss: 0.2853 - accuracy: 0.8744 - val_loss: 0.3840 - val_accuracy: 0.8301

Epoch 00003: loss improved from 0.38446 to 0.28755, saving model to LSTM_models\best_LSTM_model.h5
Epoch 4/100
116/116 [==============================] - 33s 283ms/step - loss: 0.2037 - accuracy: 0.9111 - val_loss: 0.4885 - val

116/116 [==============================] - 37s 315ms/step - loss: 0.0114 - accuracy: 0.9953 - val_loss: 1.3516 - val_accuracy: 0.7868

Epoch 00036: loss did not improve from 0.01138
Epoch 37/100
116/116 [==============================] - 37s 315ms/step - loss: 0.0104 - accuracy: 0.9953 - val_loss: 1.4645 - val_accuracy: 0.7976

Epoch 00037: loss did not improve from 0.01138
Epoch 38/100
116/116 [==============================] - 37s 315ms/step - loss: 0.0133 - accuracy: 0.9931 - val_loss: 1.2329 - val_accuracy: 0.7473

Epoch 00038: loss did not improve from 0.01138
Epoch 39/100
116/116 [==============================] - 37s 316ms/step - loss: 0.0490 - accuracy: 0.9821 - val_loss: 1.2716 - val_accuracy: 0.7955

Epoch 00039: loss did not improve from 0.01138
Epoch 40/100
116/116 [==============================] - 37s 315ms/step - loss: 0.0302 - accuracy: 0.9893 - val_loss: 1.2740 - val_accuracy: 0.7900

Epoch 00040: loss did not improve from 0.01138
Epoch 41/100
116/116 [================

In [22]:
reconstructed_model = keras.models.load_model("LSTM_models/best_LSTM_model.h5")
reconstructed_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 53, 128)           1819776   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 53, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 2,099,522
Trainable params: 2,099,522
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.load_weights('LSTM_models/best_LSTM_model.h5')

y_test_hat = model.predict(test_text_vec)
confusion = confusion_matrix(np.argmax(y_test_label,axis=1), np.argmax(y_test_hat,axis=1))
class_report = classification_report(np.argmax(y_test_label, axis=1), np.argmax(y_test_hat, axis=1))
                                     
print("Confusion matrix:\n", confusion)
print("\n")
print("Classification report:\n",class_report)

perf_metrics = evaluate.performance(y_test, np.argmax(y_test_hat,axis=1), y_test_hat)
print(perf_metrics['report'])

threshold_metrics = evaluate.threshold(y_test_hat, y_test)
print(threshold_metrics)

Confusion matrix:
 [[1216  229]
 [ 155  248]]


Classification report:
               precision    recall  f1-score   support

           0       0.89      0.84      0.86      1445
           1       0.52      0.62      0.56       403

    accuracy                           0.79      1848
   macro avg       0.70      0.73      0.71      1848
weighted avg       0.81      0.79      0.80      1848


The evaluation report of classification is:
Confusion Matrix:
[[1216  229]
 [ 155  248]]
Accuracy: 0.7922077922077922
Precision: 0.519916142557652
Recall: 0.6153846153846154
F2 Score: 0.5935854475825755
AUC Score: 0.8153227952982391

{'threshold': 0.01, 'score': 0.6377204884667571, 'y_pred': array([0, 0, 0, ..., 1, 0, 0])}


In [24]:
# import seaborn as sns

# sns.distplot(y_test_hat)

### Bidirectional LSTM with CNN layer

In [25]:
DEFAULT_BATCH_SIZE = 64
DEFAULT_EPOCHS = 100

model_cnn = Sequential()
model_cnn.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
model_cnn.add(SpatialDropout1D(0.2))
model_cnn.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
model_cnn.add(Conv1D(64, 4))
model_cnn.add(GlobalMaxPool1D())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(2, activation='sigmoid'))


model_cnn.compile(optimizer = 'adam', 
              loss=keras.losses.BinaryCrossentropy(from_logits = False),
              metrics=['accuracy'])

early_stop_cnn = keras.callbacks.EarlyStopping(monitor = 'loss',
                                                  verbose = 1,
                                                  patience = 10,
                                                  mode = 'auto',
                                                  restore_best_weights = True
                                                 )

checkpoint_cnn = ModelCheckpoint('LSTM_models/best_LSTM_CNN_model.h5', monitor='loss', mode='auto', 
                             verbose = 1, save_best_only=True)

callbacks_list_cnn = [checkpoint_cnn, early_stop_cnn]

model_cnn.fit(x=train_text_vec,
              y=y_train_label,
              class_weight=cws,
              batch_size=DEFAULT_BATCH_SIZE,
              epochs=DEFAULT_EPOCHS,
              callbacks=callbacks_list_cnn,
              verbose=1,
              validation_data=(
                  test_text_vec,
                  y_test_label
              ))

Epoch 1/100
116/116 [==============================] - 36s 288ms/step - loss: 0.6197 - accuracy: 0.5962 - val_loss: 0.3650 - val_accuracy: 0.8398

Epoch 00001: loss improved from inf to 0.53590, saving model to LSTM_models\best_LSTM_CNN_model.h5
Epoch 2/100
116/116 [==============================] - 33s 288ms/step - loss: 0.3430 - accuracy: 0.8453 - val_loss: 0.3816 - val_accuracy: 0.8328

Epoch 00002: loss improved from 0.53590 to 0.34871, saving model to LSTM_models\best_LSTM_CNN_model.h5
Epoch 3/100
116/116 [==============================] - 35s 304ms/step - loss: 0.2461 - accuracy: 0.8889 - val_loss: 0.4249 - val_accuracy: 0.8274

Epoch 00003: loss improved from 0.34871 to 0.24780, saving model to LSTM_models\best_LSTM_CNN_model.h5
Epoch 4/100
116/116 [==============================] - 36s 311ms/step - loss: 0.1591 - accuracy: 0.9272 - val_loss: 0.5283 - val_accuracy: 0.8182

Epoch 00004: loss improved from 0.24780 to 0.16822, saving model to LSTM_models\best_LSTM_CNN_model.h5
Epoc

In [26]:
reconstructed_model_cnn = keras.models.load_model("LSTM_models/best_LSTM_CNN_model.h5")
reconstructed_model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 53, 128)           1819776   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 53, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 53, 256)           263168    
_________________________________________________________________
conv1d (Conv1D)              (None, 50, 64)            65600     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [27]:
model_cnn.load_weights('LSTM_models/best_LSTM_CNN_model.h5')

y_test_hat_cnn = model_cnn.predict(test_text_vec)
confusion_cnn = confusion_matrix(np.argmax(y_test_label,axis=1), np.argmax(y_test_hat_cnn,axis=1))
class_report_cnn = classification_report(np.argmax(y_test_label, axis=1), np.argmax(y_test_hat_cnn, axis=1))
                                     
print("Confusion matrix:\n", confusion_cnn)
print("\n")
print("Classification report:\n",class_report_cnn)

perf_metrics_cnn = evaluate.performance(y_test, np.argmax(y_test_hat_cnn,axis=1), y_test_hat_cnn)
print(perf_metrics_cnn['report'])

threshold_metrics_cnn = evaluate.threshold(y_test_hat_cnn, y_test)
print(threshold_metrics_cnn)

Confusion matrix:
 [[1267  178]
 [ 196  207]]


Classification report:
               precision    recall  f1-score   support

           0       0.87      0.88      0.87      1445
           1       0.54      0.51      0.53       403

    accuracy                           0.80      1848
   macro avg       0.70      0.70      0.70      1848
weighted avg       0.79      0.80      0.80      1848


The evaluation report of classification is:
Confusion Matrix:
[[1267  178]
 [ 196  207]]
Accuracy: 0.7976190476190477
Precision: 0.5376623376623376
Recall: 0.5136476426799007
F2 Score: 0.5182774161241863
AUC Score: 0.8162861583109378

{'threshold': 0.01, 'score': 0.6185567010309277, 'y_pred': array([1, 0, 0, ..., 0, 0, 0])}


In [28]:
# import seaborn as sns

# sns.distplot(y_test_hat_cnn)